In [17]:
import numpy as np
import nengo
import nengo_dl

transform = np.random.normal(size=(3*10**4, 3*10**4))  # Fails
# transform = np.zeros((3*10**4, 3*10**4))  # Works
print(transform.nbytes / 1e9, 'GB')  # 7.2 GB

with nengo.Network() as net:
    ens = nengo.Ensemble(transform.shape[0], 1)
    nengo.Connection(ens.neurons, ens.neurons, transform=transform)
sim = nengo_dl.Simulator(net)

ModuleNotFoundError: No module named 'nengo'

In [4]:
import tensorflow.compat.v2 as tf 
import tensorflow_datasets as tfds 
tf.enable_v2_behavior() 
from tensorflow.python.framework.ops import disable_eager_execution 
disable_eager_execution()

In [5]:
%cd /Users/jye98/Desktop/dataset2

/Users/jye98/Desktop/dataset2


In [6]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
import os

In [7]:
image_datas = glob.glob('./**/*.png')
class_name = ["StyleGAN3", "RealImage"]
dic = {"StyleGAN3":0, "RealImage":1}

In [18]:
x = []
y = []
for imagename in image_datas:
    image = Image.open(imagename)
    image = image.resize((300,300))
    image = np.array(image)
    x.append(image)
    label = imagename.split('/')[1]
    label = dic[label]
    y.append(label)

In [19]:
x = np.array(x)
y = np.array(y)

In [20]:
train_images, test_images, train_labels, test_labels = train_test_split(x, y, test_size=0.2, shuffle= True, random_state=777)
train_labels = train_labels[..., tf.newaxis]
test_labels = test_labels[..., tf.newaxis]

test_images.shape, test_labels.shape, train_images.shape, train_labels.shape

((312, 300, 300, 3), (312, 1), (1244, 300, 300, 3), (1244, 1))

In [21]:
#training set의 각 class 별 image 수 확인
unique, counts = np.unique(np.reshape(train_labels, (1244,)), axis=-1, return_counts=True)
dict(zip(unique, counts))

{0: 582, 1: 662}

In [22]:
N_TRAIN = train_images.shape[0]
N_TEST = test_images.shape[0] 

In [23]:
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

In [24]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(1244, 300, 300, 3) (1244, 2)
(312, 300, 300, 3) (312, 2)


In [25]:
learning_rate = 0.01
N_EPOCHS = 10
N_BATCH = 50
N_CLASS = 5

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(buffer_size=1244).batch(N_BATCH).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(N_BATCH)

In [27]:
import os
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.applications import Xception
import tensorflow as tf
tf.keras.applications.Xception(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000
)

import matplotlib.pyplot as plt
import math

Instructions for updating:
Colocations handled automatically by placer.
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-25 22:58:10.930796: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-25 22:58:10.931217: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-25 22:58:12.954634: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-25 22:58:12.962152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 22:58:15.368523: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 22:58:16.051745: I tensorflow/core/grappler/o

In [28]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import binary_crossentropy
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

import matplotlib.pyplot as plt
import math

In [35]:
xception_base = Xception(weights='imagenet',
                            include_top = False,
                            input_shape = (300, 300, 3))

model = models.Sequential()
model.add(xception_base)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

for layer in xception_base.layers:
    layer.trainable = False

for layer in xception_base.layers[15:]:
    layer.trainable = True
    
loss = binary_crossentropy
model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=3e-4, beta_2=0.999, epsilon=1e-08), metrics=['accuracy'])

xception_base.summary()

2022-05-25 23:03:44.850937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:03:49.181266: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:03:50.754722: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [36]:
steps_per_epoch = N_TRAIN//N_BATCH
validation_steps = N_TEST//N_BATCH
print(steps_per_epoch, validation_steps)

24 6


In [37]:
history = model.fit(train_dataset, epochs=N_EPOCHS,steps_per_epoch=steps_per_epoch,
                    validation_data = test_dataset, validation_steps=validation_steps)

2022-05-25 23:04:01.080628: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:04:03.869134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:04:04.341445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:04:31.451494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:05:01.175391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:05:04.452357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:05:05.539280: I tensorflow/core/grappler/optimizers/cust

Train on 24 steps, validate on 6 steps
Epoch 1/10


2022-05-25 23:05:12.318363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-25 23:05:12.763591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


14/24 [================>.............] - ETA: 59s - batch: 6.5000 - size: 1.0000 - loss: 6.9220 - accuracy: 0.5000 